In [1]:
# import dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression

In [3]:
# import data
redfin_df = pd.read_csv("./cleaned_data.csv")
redfin_df.head()

,full_address,beds,baths,year_built,sq_ft_interior,sq_ft_lot,price,lat,long,zip_code
0,2326 LUAU ST MESQUITE TX 75150,3,1.5,1966,1356,8303,241183,32.798774,-96.649097,75150
1,1737 HIGHLAND ST MESQUITE TX 75149,3,1.0,1955,1454,7501,218014,32.787427,-96.609647,75149
2,2900 SIDNEY DR MESQUITE TX 75150,3,2.0,1972,1392,7196,241089,32.817247,-96.644125,75150
3,1507 RICHARD ST MESQUITE TX 75149,3,1.0,1957,1313,7501,205940,32.787413,-96.616575,75149
4,806 BRANDON DR SEAGOVILLE TX 75159,5,2.0,2004,1504,8407,275611,32.661589,-96.547004,75159


In [4]:
redfin_copy_df = redfin_df.copy()

In [5]:
# drop columns
redfin_copy_df = redfin_copy_df.drop(columns=['full_address', 'lat', 'long'])
redfin_copy_df.head()

,beds,baths,year_built,sq_ft_interior,sq_ft_lot,price,zip_code
0,3,1.5,1966,1356,8303,241183,75150
1,3,1.0,1955,1454,7501,218014,75149
2,3,2.0,1972,1392,7196,241089,75150
3,3,1.0,1957,1313,7501,205940,75149
4,5,2.0,2004,1504,8407,275611,75159


In [6]:
redfin_copy_df.nunique()

beds                 12
baths                22
year_built          124
sq_ft_interior     3944
sq_ft_lot          5680
price             31895
zip_code             82
dtype: int64

In [7]:
redfin_copy_df.dtypes

beds                int64
baths             float64
year_built          int64
sq_ft_interior      int64
sq_ft_lot           int64
price               int64
zip_code            int64
dtype: object

In [8]:
# zip needs to be treated as categorical
redfin_copy_df['zip_code'] = redfin_copy_df['zip_code'].astype(str)
redfin_copy_df.dtypes

beds                int64
baths             float64
year_built          int64
sq_ft_interior      int64
sq_ft_lot           int64
price               int64
zip_code           object
dtype: object

In [9]:
# convert categorical data
redfin_copy_df = pd.get_dummies(redfin_copy_df)
redfin_copy_df.head()

,beds,baths,year_built,sq_ft_interior,sq_ft_lot,price,zip_code_75001,zip_code_75006,zip_code_75007,zip_code_75019,...,zip_code_75240,zip_code_75241,zip_code_75243,zip_code_75244,zip_code_75246,zip_code_75248,zip_code_75249,zip_code_75252,zip_code_75253,zip_code_75254
0,3,1.5,1966,1356,8303,241183,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,3,1.0,1955,1454,7501,218014,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,2.0,1972,1392,7196,241089,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,1.0,1957,1313,7501,205940,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,2.0,2004,1504,8407,275611,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
# split preprocessed data into features and targets
y = redfin_copy_df['price'].values
X = redfin_copy_df.drop(columns='price').values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [11]:
# create the scaler
scaler = StandardScaler()

# fit the scaler
X_scaler = scaler.fit(X_train)

# scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [18]:
# create custom activation function for evaluating accuracy within a percentage range
def accuracy_calculation(y_true, y_pred, threshold=0.05):
    error = tf.abs(y_true - y_pred)
    return tf.where(error < threshold, y_pred, 0)

In [26]:
input_dim = X_train.shape[1]

nn = tf.keras.models.Sequential()

# first hidden layer
nn.add(tf.keras.layers.Dense(units=10, activation='relu', input_dim=input_dim))

# second hidden layer
nn.add(tf.keras.layers.Dense(units=5, activation='relu'))

# third hidden layer
nn.add(tf.keras.layers.Dense(units=5, activation='softplus')) # linear relu softplus tanh

nn.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_15 (Dense)            (None, 10)                880       
                                                                 
 dense_16 (Dense)            (None, 5)                 55        
                                                                 
 dense_17 (Dense)            (None, 5)                 30        
                                                                 
Total params: 965 (3.77 KB)
Trainable params: 965 (3.77 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [37]:
# compile the model
nn.compile(loss='mean_absolute_percentage_error', optimizer='adam', metrics=[tf.keras.metrics.RootMeanSquaredError(), 'mae'])

In [38]:
fit_model = nn.fit(X_train_scaled, y_train, epochs = 50)

Epoch 1/50
779/779 [==============================] - 3s 2ms/step - loss: 10.3230 - root_mean_squared_error: 337078.5938 - mae: 60298.0078
Epoch 2/50
779/779 [==============================] - 2s 2ms/step - loss: 10.3207 - root_mean_squared_error: 337118.6250 - mae: 60315.5977
Epoch 3/50
779/779 [==============================] - 3s 3ms/step - loss: 10.3192 - root_mean_squared_error: 337090.0625 - mae: 60274.4180
Epoch 4/50
779/779 [==============================] - 2s 3ms/step - loss: 10.3179 - root_mean_squared_error: 337044.4688 - mae: 60279.3047
Epoch 5/50
779/779 [==============================] - 2s 2ms/step - loss: 10.3149 - root_mean_squared_error: 337082.2812 - mae: 60262.6992
Epoch 6/50
779/779 [==============================] - 2s 2ms/step - loss: 10.3138 - root_mean_squared_error: 336982.8125 - mae: 60241.9258
Epoch 7/50
779/779 [==============================] - 2s 2ms/step - loss: 10.3112 - root_mean_squared_error: 336968.4062 - mae: 60228.7852
Epoch 8/50
779/779 [=======